In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import io
from datetime import datetime,timedelta
import pandas as pd
import os
path = os.getcwd()

if path[1:5] == "Home":
    %run /Home/siv30/wad005/master/GRB/src/paths/uib.ipynb
    %run /Home/siv30/wad005/master/GRB/src/read_matlab_file.ipynb
    %run /Home/siv30/wad005/master/GRB/src/convert_erg.ipynb
    %run /Home/siv30/wad005/master/GRB/src/safety_time.ipynb
    path_m = '/Home/siv30/wad005/master/GRB/MATLAB/david_sarria/'
else:
    %run /Users/andreas/phys/master/GRB/src/paths/mac.ipynb # Mac paths
    %run /Users/andreas/phys/master/GRB/src/read_matlab_file.ipynb
    %run /Users/andreas/phys/master/GRB/src/convert_erg.ipynb
    %run /Users/andreas/phys/master/GRB/src/safety_time.ipynb
    path_m = '/Users/andreas/phys/master/GRB/MATLAB/david_sarria/'


all UiB paths loaded

all UiB paths loaded
read_matlab_file.ipynb loaded successfully!

all UiB paths loaded
read_matlab_file.ipynb loaded successfully!
convert_erg.ipynb loaded successfully!

all UiB paths loaded
read_matlab_file.ipynb loaded successfully!

all UiB paths loaded
read_matlab_file.ipynb loaded successfully!
convert_erg.ipynb loaded successfully!
safety_time.ipynb loaded successfully!


In [1]:
def make_adr(df):
    dau, det = df['dau'].to_numpy().astype(np.int32), df['det'].to_numpy().astype(np.int32)
    if np.any(dau == 0) or np.any(det == 0):
        dau += 1
        det += 1
    adr = (dau-1)*3 + det
    df['adr'] = adr
    df['dau'] = dau
    df['det'] = det
    return df

In [1]:
def harvest_data(fn=None,T0=None):
    data = scipy.io.loadmat(path_m+fn)['out_data']
    df = pd.DataFrame(data=data)
    df = df.drop_duplicates()
    df.rename(columns={0:'year',1:'month',2:'day',3:'hour',4:'minute',5:'second',6:'erg',7:'dau',8:'det'},inplace=True)
    df = df.convert_dtypes()
    dt_column = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute', 'second']].astype(dtype=str))
    df['datetime'] = dt_column
    df = df.drop(columns=['year', 'month', 'day', 'hour', 'minute', 'second'])
    T0 = datetime.fromisoformat(T0)
    tus = df['datetime']-T0
    df['tus'] = tus
    df['tus'] = df['tus'].dt.total_seconds().astype(np.float64)
    df['dau'] = df['dau'].to_numpy()+1 #index starting now at 1
    df['det'] = df['det'].to_numpy()+1 #index starting now at 1
    df = make_adr(df)
    
    df.sort_values(by=['adr','tus'],inplace=True,ignore_index=True)
    print('dau/det starting from index 1')
    print('tus array given in seconds')
    
    tus,erg,dau,det = df['tus'].to_numpy(),df['erg'].to_numpy(),df['dau'].to_numpy(),df['det'].to_numpy()
    keV,abc = convert_erg(erg,dau,det,T0)
    df['keV'] = keV
    df['tus'] = df['tus'].to_numpy()*1e6 #converting to us

    st,dt = get_st(df),get_dt(df)
    df['st']=st
    df['dt']=dt

    flag = get_flag(df)
    df['flag']=flag
    print('safety time: ',np.round((len(np.where(flag==True)[0])/len(flag))*100,3), '% removed')
    df.drop(labels=np.where(flag==True)[0],inplace=True)
    df.sort_values(by='tus',inplace=True,ignore_index=True)
    df['tus'] = df['tus'].to_numpy()/1e6 #converting to s
    print('sorted by tus (s)')
    df = df.reindex(columns=['datetime','tus','adr','dau','det','erg','keV','st','dt','flag'])
    return df